In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from bs4 import BeautifulSoup
from urllib.parse import quote
import requests, re
import time
from tqdm import tqdm
import pickle
import sqlite3
import csv  
from concurrent.futures import ThreadPoolExecutor, as_completed
import gzip, json
import pyarrow as pa, pyarrow.parquet as pq
from io import BytesIO
import zlib

In [2]:
# DOWNLOAD OPEN LIBRARY:

url = 'https://openlibrary.org/search.json?q={category}&language=eng&page={page}'
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
else:
    print(f'System error accessing the page. Status code: {response.status_code}')

In [4]:
# 1. PREPARE DE DATA WE NEED:

categories = [
    "fiction", "historical fiction", "mystery", "thriller", "romance", "fantasy", "romantsy",
    "science fiction", "horror", "young adult", "nonfiction", "memoir", "autobiography", "history",
    "biography", "humor", "gay", "lgbt", "queer", "paranormal", "historical romance", "contemporary",
    "classic", "comics", "manga", "true crime", "poetry", "graphic novels", "adventure"
]

TARGET_BOOKS    = 60000   # we need 60.000 rows 
OVERSHOOT_FACTOR = 2      # for duplicates, scrape the double raw
OL_LIMIT         = 1000   # maximum allowed per request to OpenLibrary


# 2.  EXTRACT JSON FROM OPENLIBRARY WITH PAGINATION (limit+offset)

def get_ol_books_by_category_offset(category, max_books, batch=OL_LIMIT):
    docs, offset = [], 0
    while len(docs) < max_books:
        url = (
            "https://openlibrary.org/search.json?"
            f"q={category}&language=eng&limit={batch}&offset={offset}"
        )
        resp = requests.get(url)
        if resp.status_code != 200:
            print(f"Error {resp.status_code} en categoría='{category}', offset={offset}")
            break

        batch_docs = resp.json().get("docs", [])
        if not batch_docs:
            break

        docs.extend(batch_docs)
        offset += batch
        time.sleep(1)  # do not boom the API

    return docs[:max_books]


# 3. ENRICH EACH DOC ​​WITH /works/{key}.json (series, awards, topics)

def enrich_work(doc):
    rec = {
        "series": None,
        "awards": None,
        "ol_genres": ", ".join(doc.get("subject", []))
    }
    key = doc.get("key")
    if not key:
        return rec

    try:
        r = requests.get(f"https://openlibrary.org{key}.json", timeout=5)
        if r.status_code == 200:
            w = r.json()
            if w.get("series"):
                rec["series"] = ", ".join(w["series"])
            if w.get("awards"):
                rec["awards"] = ", ".join(w["awards"])
            if w.get("subjects"):
                rec["ol_genres"] += ", " + ", ".join(w["subjects"])
    except:
        pass

    return rec


# 4. CHECK GOOGLE BOOKS BY THE ISBN

def fetch_gb(isbn):
    if not isbn:
        return {}
    try:
        r = requests.get(f"https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}", timeout=5)
        info = r.json().get("items", [{}])[0].get("volumeInfo", {})
        return {
            "publisher":     info.get("publisher"),
            "pageCount":     info.get("pageCount"),
            "averageRating": info.get("averageRating"),
            "ratingsCount":  info.get("ratingsCount"),
            "printType":     info.get("printType"),
            "gb_genres":     ", ".join(info.get("categories", []))
        }
    except:
        return {}


# 5. MAIN FLOW

def main():
    raw_docs = []
    # We multiply raw by OVERSHOOT_FACTOR to ensure 60k after dedup
    per_cat = int(np.ceil(TARGET_BOOKS * OVERSHOOT_FACTOR / len(categories)))

    # 5.1. Extract raw_docs
    for cat in categories:
        print(f"Extract until {per_cat} docs of '{cat}'…")
        docs = get_ol_books_by_category_offset(cat, max_books=per_cat)
        raw_docs.extend(docs)
        if len(raw_docs) >= TARGET_BOOKS * OVERSHOOT_FACTOR:
            break

    print(f"Extracted Raw: {len(raw_docs)} (objetive raw ~{TARGET_BOOKS*OVERSHOOT_FACTOR})")
    # cut a raw máximo
    raw_docs = raw_docs[: TARGET_BOOKS * OVERSHOOT_FACTOR]

    # 5.2. Enrich works.json in parallel
    print("🔍 Enriching works/{key}.json …")
    with ThreadPoolExecutor(max_workers=20) as exe:
        enriched = list(tqdm(exe.map(enrich_work, raw_docs), total=len(raw_docs)))

    # 5.3. Build base and deduplicate the DataFrame
    base = []
    for doc, extra in zip(raw_docs, enriched):
        base.append({
            "title":               doc.get("title"),
            "original_title":      doc.get("title"),
            "authors":             ", ".join(doc.get("author_name", [])),
            "cover_url":           f"https://covers.openlibrary.org/b/id/{doc.get('cover_i')}-L.jpg"
                                     if doc.get("cover_i") else None,
            "first_publish_year":  doc.get("first_publish_year"),
            "original_language":   ", ".join(doc.get("language", [])),
            "isbn":                ", ".join(doc.get("isbn", [])),
            "edition_count":       doc.get("edition_count"),
            **extra
        })
    df = pd.DataFrame(base)
    before = len(df)
    df.drop_duplicates(subset=["isbn", "title"], inplace=True)
    print(f"After deduplicate: {before} → {len(df)}")

    # 5.4. Google Books in parallel
    print("Consulting Google Books by ISBN …")
    isbns = df["isbn"].fillna("").str.split(",").str[0].tolist()
    with ThreadPoolExecutor(max_workers=20) as exe:
        gb = list(tqdm(exe.map(fetch_gb, isbns), total=len(isbns)))
    gb_df = pd.DataFrame(gb)
    df = pd.concat([df.reset_index(drop=True), gb_df], axis=1)

    # 5.5. Placeholders and reordering columns
    for col in ["physical_format", "reviews", "stetings", "characters"]:
        df[col] = None

    final_cols = [
        "title","original_title","authors","series","cover_url",
        "first_publish_year","publisher","original_language","isbn",
        "ol_genres","gb_genres","averageRating","ratingsCount",
        "pageCount","printType","physical_format","awards",
        "edition_count","reviews","setting","charaters"
    ]
    df = df.reindex(columns=final_cols)

    print(f"Final DataFrame: {df.shape[0]} rows × {df.shape[1]} columns")

    # 5.6. Export
    df.to_csv("openlibrary_60000_books.csv", index=False, encoding="utf-8-sig")
    print("Saved in openlibrary_60000_books.csv")

if __name__ == "__main__":
    main()

Extract until 4138 docs of 'fiction'…
Extract until 4138 docs of 'historical fiction'…
Extract until 4138 docs of 'mystery'…
Extract until 4138 docs of 'thriller'…
Extract until 4138 docs of 'romance'…
Extract until 4138 docs of 'fantasy'…
Extract until 4138 docs of 'romantsy'…
Extract until 4138 docs of 'science fiction'…
Extract until 4138 docs of 'horror'…
Extract until 4138 docs of 'young adult'…
Extract until 4138 docs of 'nonfiction'…
Extract until 4138 docs of 'memoir'…
Extract until 4138 docs of 'autobiography'…
Extract until 4138 docs of 'history'…
Extract until 4138 docs of 'biography'…
Extract until 4138 docs of 'humor'…
Extract until 4138 docs of 'gay'…
Extract until 4138 docs of 'lgbt'…
Extract until 4138 docs of 'queer'…
Extract until 4138 docs of 'paranormal'…
Extract until 4138 docs of 'historical romance'…
Extract until 4138 docs of 'contemporary'…
Extract until 4138 docs of 'classic'…
Extract until 4138 docs of 'comics'…
Extract until 4138 docs of 'manga'…
Extract unt

100%|██████████| 113359/113359 [2:29:45<00:00, 12.62it/s] 


After deduplicate: 113359 → 77611
Consulting Google Books by ISBN …


100%|██████████| 77611/77611 [00:00<00:00, 454640.10it/s]


Final DataFrame: 77611 rows × 21 columns
Saved in openlibrary_60000_books.csv


In [41]:
# TAKE A LOOK TO THE DATABASE:

df_books = pd.read_csv('Data Base/openlibrary_60000_books.csv')
df_books.head()

,title,original_title,authors,series,cover_url,first_publish_year,publisher,original_language,isbn,ol_genres,...,averageRating,ratingsCount,pageCount,printType,physical_format,awards,edition_count,reviews,setting,charaters
0,Foundation,Foundation,Isaac Asimov,NaN,https://covers.openlibrary.org/b/id/14612610-L...,1951.0,NaN,"rus, eng, ita, por, fre, ger, kor, spa, chi",NaN,", Psychohistory, Open Library Staff Picks, Lif...",...,NaN,NaN,NaN,NaN,NaN,NaN,97,NaN,NaN,NaN
1,Ficciones,Ficciones,Jorge Luis Borges,NaN,https://covers.openlibrary.org/b/id/10832290-L...,1945.0,NaN,"spa, ita, eng, por, fre",NaN,", Anachronisms, speculative fiction, subjectiv...",...,NaN,NaN,NaN,NaN,NaN,NaN,78,NaN,NaN,NaN
2,Dracula,Dracula,Bram Stoker,NaN,https://covers.openlibrary.org/b/id/12216503-L...,1897.0,NaN,"tur, gle, cat, pol, chi, eng, ger, ita, spa, g...",NaN,", English literature, Fiction, Horror, Horror ...",...,NaN,NaN,NaN,NaN,NaN,NaN,730,NaN,NaN,NaN
3,The Last Man,The Last Man,Mary Shelley,NaN,https://covers.openlibrary.org/b/id/882662-L.jpg,1826.0,NaN,"eng, ger, ita",NaN,", Fiction, Plague, Twenty-first century, End o...",...,NaN,NaN,NaN,NaN,NaN,NaN,383,NaN,NaN,NaN
4,Beloved,Beloved,Toni Morrison,NaN,https://covers.openlibrary.org/b/id/8261367-L.jpg,1987.0,NaN,"rus, eng, ita, swe, tur, fre, jpn, ger, kor, s...",NaN,", African American History, Ohio, History, 19t...",...,NaN,NaN,NaN,NaN,NaN,NaN,104,NaN,NaN,NaN


In [42]:
df_books.shape

(77611, 21)

In [43]:
df_books.describe().T

,count,mean,std,min,25%,50%,75%,max
series,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first_publish_year,77381.0,1989.523888,52.396212,0.0,1985.0,2007.0,2017.0,2312.0
publisher,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
isbn,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gb_genres,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
averageRating,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ratingsCount,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pageCount,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
printType,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
physical_format,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
df_books.dtypes

title                  object
original_title         object
authors                object
series                float64
cover_url              object
first_publish_year    float64
publisher             float64
original_language      object
isbn                  float64
ol_genres              object
gb_genres             float64
averageRating         float64
ratingsCount          float64
pageCount             float64
printType             float64
physical_format       float64
awards                float64
edition_count           int64
reviews               float64
setting               float64
charaters             float64
dtype: object

In [45]:
df_books.nunique()

title                 77611
original_title        77611
authors               47503
series                    0
cover_url             43402
first_publish_year      499
publisher                 0
original_language      4755
isbn                      0
ol_genres              1550
gb_genres                 0
averageRating             0
ratingsCount              0
pageCount                 0
printType                 0
physical_format           0
awards                    0
edition_count           564
reviews                   0
setting                   0
charaters                 0
dtype: int64

In [46]:
df_books['first_publish_year'].value_counts()

first_publish_year
2021.0    3828
2020.0    3167
2019.0    3134
2017.0    2939
2018.0    2720
          ... 
1608.0       1
1571.0       1
1530.0       1
1654.0       1
2312.0       1
Name: count, Length: 499, dtype: int64

In [15]:
META_URL = "https://archive.org/metadata/ol_dump_editions_latest"
m = requests.get(META_URL, timeout=30)
m.raise_for_status()
files = m.json()["files"]
# buscamos el nombre versionado, e.g. ol_dump_editions_2025-07-10.txt.gz
fn = next(f["name"] for f in files
          if f["name"].startswith("ol_dump_editions_") and f["name"].endswith(".txt.gz"))

DL_URL = f"https://archive.org/download/ol_dump_editions_latest/{fn}"
print("✓ Descargando edición dump desde:", DL_URL)

# 2) Parámetros de procesado
TARGET     = 60000      # número de libros ocio a capturar
BATCH_SIZE = 5000       # cuántos registros acumular antes de escribir Parquet
OUTPUT_PQ  = "books_leisure_60k.parquet"

OCIO    = {"fiction","fantasy","mystery","thriller","romance",
           "horror","adventure","sci-fi","young adult"}
NO_OCIO = {"academic","textbook","philosophy","biography"}

def is_leisure(subjects):
    gs = {s.strip().lower() for s in subjects or []}
    return bool(gs & OCIO) and not bool(gs & NO_OCIO)

def extract_year(d):
    m = re.search(r"(\d{4})", str(d))
    return m.group(1) if m else ""

# 3) Baja y descomprime en streaming, procesando línea a línea
resp = requests.get(DL_URL, stream=True, timeout=60)
resp.raise_for_status()
decomp = gzip.GzipFile(fileobj=BytesIO(resp.raw.read(1024*1024)), mode="rb")

writer = None
batch = []
count = 0
pbar = tqdm(total=TARGET, desc="Ediciones ocio")

for raw in decomp:
    if count >= TARGET:
        break
    try:
        ed = json.loads(raw)
    except:
        continue

    # filtrar ediciones (tienen isbn_13)
    isbns = ed.get("isbn_13") or []
    if not isbns:
        continue

    # filtrar ocio
    subs = ed.get("subjects") or []
    if not is_leisure(subs):
        continue

    # arma tu registro de 20 cols
    rec = {
      "title":             ed.get("title",""),
      "original_title":    ed.get("subtitle","") or ed.get("title",""),
      "authors":           "; ".join(a.get("name","") for a in ed.get("authors",[])),
      "series":            "; ".join(w.get("key","") for w in ed.get("works",[])),
      "cover_url":         (f"https://covers.openlibrary.org/b/id/{ed.get('covers',[None])[0]}-L.jpg"
                             if ed.get("covers") else ""),
      "first_publish_year":extract_year(ed.get("publish_date","")),
      "publisher":         (ed.get("publishers") or [""])[0],
      "original_language": (ed.get("languages") or [""])[0].split("/")[-1],
      "isbn":              isbns[0],
      "ol_genres":         "; ".join(subs),
      "gb_genres":         "",    # o mapear tras
      "averageRating":     None,  # merge con BX si lo añades
      "ratingsCount":      None,
      "pageCount":         ed.get("number_of_pages",None),
      "printType":         "Book",
      "physical_format":   ed.get("physical_format",""),
      "awards":            "",    # extraer con regex/NLP de subjects
      "edition_count":     ed.get("edition_count",None),
      "reviews":           None,
      "setting":           "",    # derivar después
      "characters":        ""
    }

    batch.append(rec)
    count += 1
    pbar.update(1)

    # cada BATCH_SIZE registros, escribe un row‐group
    if len(batch) >= BATCH_SIZE:
        df = pd.DataFrame(batch)
        tbl = pa.Table.from_pandas(df)
        if writer is None:
            writer = pq.ParquetWriter(OUTPUT_PQ, tbl.schema)
        writer.write_table(tbl)
        batch.clear()

# vuelca remanentes y cierra
if batch:
    df = pd.DataFrame(batch)
    tbl = pa.Table.from_pandas(df)
    if writer is None:
        writer = pq.ParquetWriter(OUTPUT_PQ, tbl.schema)
    writer.write_table(tbl)
if writer:
    writer.close()

pbar.close()
print(f"✅ Parquet listo: {OUTPUT_PQ} con {count} libros de ocio.")

KeyError: 'files'